# Fine-tuned ALBERT Model for Constructiveness Detection in Steam Reviews
## *Sentiment-Analysis of Videogame Reviews on the Platform ”Steam” with a Focus on the Detection and Classification of <b>Constructiveness</b>*
---
### <u>NOTEBOOK **4**/5</u>: This Notebook tests the inference using the fine-tuned model.

In [ ]:
# Package Installs
!pip install -U pip
!pip install datasets transformers accelerate scikit-learn tensorboard wandb

# Installs TensorFlow from the NVIDIA repository
!pip install nvidia-pyindex
!pip install nvidia-tensorflow[horovod]

# Installs PyTorch with CUDA 12.4 support
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# Specific version of protobuf to maintain compatibility with TensorFlow
!pip uninstall -y protobuf
!pip install protobuf==3.20.*

In [ ]:
# Imports
import pandas as pd
#from google.colab import files
import IPython
import io
from datasets import Dataset, DatasetDict
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, pipeline
from sklearn.model_selection import train_test_split
import tensorflow as tf
import datetime
import wandb
import torch
import re

In [ ]:
import sys

# Flag that automaticaly configures the notebook depending on if it is run in google colab or locally
RUNNING_IN_GOOGLE_COLAB = "google.colab" in sys.modules
print("Running in Google Colab" if RUNNING_IN_GOOGLE_COLAB else "Running locally")

Running in Google Colab


In [ ]:
# System Information
# Prints all sorts of system information (CPU, GPU, RAM, CUDA Installed, CUDA Version, RAM) about the google colab runtime
print("\033[1m" + "GPU Information" + "\033[0m")
print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ")
!nvidia-smi
print()
print("\033[1m" + "CPU Information" + "\033[0m")
print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ")
!cat /proc/cpuinfo
print()
print("\033[1m" + "Memory Information" + "\033[0m")
print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ")
!cat /proc/meminfo
print()
print("\033[1m" + "NVidia CUDA Information" + "\033[0m")
print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ")
!nvcc --version
print()
print("\033[1m"+ "CUDA Installation Check"+ "\033[0m")
print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ")

import torch
torch.cuda.is_available()
try:
  print("CUDA Installation Check Result: " + "\033[3m" + torch.cuda.get_device_name(0) + "\033[3m")
except:
  if RUNNING_IN_GOOGLE_COLAB:
    print("No GPU found. You might be connected to a CPU runtime in Google Colab.")
  else:
    print("No GPU found.")

In [ ]:
model_name = "bert-base-uncased"
model_checkpoint = f'/home/samuel/kaggle/finetuned_models/{model_name}-finetuned-steam-reviews'

In [ ]:
model_name = "abullard1/roberta-steam-review-constructiveness-classifier"
model_checkpoint = model_name

In [ ]:
# Loads the model from the checkpoint
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

In [ ]:
steam_review_constructivity_classifier = pipeline(
    "text-classification",
    model=model_checkpoint,
    tokenizer=model_checkpoint,
    device=0  # Uses the first GPU (device=0)
)

In [ ]:
example_text_1 = "Review: Loading times were too long. Unfortunately the game was lagging quite a bit on my PC and I frequently faced connection issues while playing., Playtime: 100, Voted Up: False, Upvotes: 12, Votes Funny: 0"

In [ ]:
example_text_2 = "Review: This game sucks ass. On God frfr., Playtime: 2, Voted Up: False, Upvotes: 0, Votes Funny: 1"

In [ ]:
example_text_3 = "Review: I think this is a great game but it still has some room for improvement., Playtime: 12, Voted Up: True, Upvotes: 1, Votes Funny: 0"

In [ ]:
example_text_4 = "Bad. Really bad. Kinda., Playtime: 4, Voted Up: False, Upvotes: 0, Votes Funny: 0"

In [ ]:
example_text_5 = "Good. Really good. Kinda., Playtime: 4, Voted Up: False, Upvotes: 0, Votes Funny: 0"

In [ ]:
def map_label(label):
  if label == "LABEL_0":
    return "[NOT CONSTRUCTIVE]"
  if label == "LABEL_1":
    return "[CONSTRUCTIVE]"

In [ ]:
def extract_label(result_dict):
  return result_dict[0]["label"]

In [ ]:
result_1 = steam_review_constructivity_classifier(example_text_1)
result_2 = steam_review_constructivity_classifier(example_text_2)
result_3 = steam_review_constructivity_classifier(example_text_3)
result_4 = steam_review_constructivity_classifier(example_text_4)
result_5 = steam_review_constructivity_classifier(example_text_5)

print(f'Review 1: \n{example_text_1}\n{map_label(extract_label(result_1))}\n\n')
print(f'Review 2: \n{example_text_2}\n{map_label(extract_label(result_2))}\n\n')
print(f'Review 3: \n{example_text_3}\n{map_label(extract_label(result_3))}\n\n')
print(f'Review 4: \n{example_text_4}\n{map_label(extract_label(result_4))}\n\n')
print(f'Review 5: \n{example_text_5}\n{map_label(extract_label(result_5))}\n\n')

Review 1: 
Review: Loading times were too long. Unfortunately the game was lagging quite a bit on my PC and I frequently faced connection issues while playing., Playtime: 100, Voted Up: False, Upvotes: 12, Votes Funny: 0
[CONSTRUCTIVE]


Review 2: 
Review: This game sucks ass. On God frfr., Playtime: 2, Voted Up: False, Upvotes: 0, Votes Funny: 1
[NOT CONSTRUCTIVE]


Review 3: 
Review: I think this is a great game but it still has some room for improvement., Playtime: 12, Voted Up: True, Upvotes: 1, Votes Funny: 0
[CONSTRUCTIVE]


Review 4: 
Bad. Really bad. Kinda., Playtime: 4, Voted Up: False, Upvotes: 0, Votes Funny: 0
[NOT CONSTRUCTIVE]


Review 5: 
Good. Really good. Kinda., Playtime: 4, Voted Up: False, Upvotes: 0, Votes Funny: 0
[NOT CONSTRUCTIVE]


